In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import torch
import torch.nn.functional as F
import torch.nn as nn
import os

import numpy as np
import matplotlib.pyplot as plt

from NormalizingFlows.src.train import train_forward
from NormalizingFlows.src.scores import log_likelihood
from NormalizingFlows.src.utils import update_device, load_best_model, load_checkpoint_model

from NormalizingFlows.src.flows import *
from NormalizingFlows.src.data.variational.linear_regression import LinearRegression
from NormalizingFlows.src.importance_sampling import *


In [3]:
def set_visible_devices(*devices: int) -> None:
    '''Utility to set visible Cuda devices.

    Parameters
    ----------
    devices : List[int]
        Index of cuda devices to make available for use.
    '''
    assert all([d >= 0 for d in devices]), f"Not all devices are CUDA devices!"
    os.environ['CUDA_VISIBLE_DEVICES'] = ",".join([str(i) for i in devices])
    

def set_devices(*devices: int):
    '''Utility to set Cuda device(s).

    Parameters
    ----------
    devices : List[int]
        Index of cuda devices to make available for use.

    Returns
    -------
    torch.device or List[torch.device] of currently available CUDA devices.
    '''
    assert len(devices) > 0, f'Device list is empty, no devices set.'
    if len(devices) == 1:
        if devices[0] >= 0:
            set_visible_devices(devices[0])
            return torch.device(0)
        else:
            return torch.device('cpu')

    else:
        set_visible_devices(*devices)
        return [torch.device(i) for i in range(len(devices))]

In [4]:
device = set_devices(6) #torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
device_cpu = torch.device("cpu")

In [5]:
dataset = LinearRegression()
dim_input = dataset.dim_input
num_trans = 8
dim_hidden = [300,300,300]

flows = {}
flow_forward = True


epochs = 10
batch_size = 32
batches = 2000
num_exp = 3

In [6]:
f_a = lambda x: torch.abs(torch.arcsinh(x))

two = torch.tensor(2)
def softsaturate(a):
    neg = 2 * (a < 0) * F.softplus(a)
    pos = (a >= 0) * (torch.arcsinh(a) + 2 * torch.log(two))
    return neg + pos

In [7]:
#Mean_field

name = 'Mean field lin_reg_exp' 
flows[name] = []
for i in range(num_exp):
    transformations = create_constant_trans(num_trans, dim_input, flow_forward, a_param=F.softplus)
    mean_field = create_flows_with_identity(dim_input, transformations, flow_forward) 
    flows[name] += [mean_field]
    flows[name][-1].name = f'{name} {i}'

In [8]:
#Affine
a_param = torch.sigmoid

name = 'Affine lin_reg_exp iar softplus'
flows[name] = []
for i in range(num_exp):
    transformations = create_affine_trans(num_trans, flow_forward, a_param = a_param)
    aff_ar_alt = create_flows_with_IAR(dim_input, dim_hidden, transformations, 'alternate', flow_forward )
    flows[name] += [aff_ar_alt]
    flows[name][-1].name = f'{name} {i}'


In [9]:
#ContinuousPiecewiseAffineAffine
a_param = f_a
c_param = torch.sigmoid

name = 'ContinuousPiecewiseAffineAffine lin_reg_exp iar'
flows[name] = []
for i in range(num_exp):
    transformations = create_affinecontinuous_trans(num_trans, flow_forward, a_param=a_param, c_param=c_param)
    affconpiec_coup_alt = create_flows_with_IAR(dim_input, dim_hidden, transformations, 'alternate', flow_forward)
    flows[name] += [affconpiec_coup_alt]
    flows[name][-1].name = f'{name} {i}'

In [10]:
#Alt. Lin. Aff.Con
a_param = f_a
c_param = torch.sigmoid 

name = 'Alternating Linear_AffineContinuous lin_reg_exp'
flows[name] = []
for i in range(num_exp):
    transformations = create_alt_linear_affinecontinuous_trans(num_trans, dim_input, flow_forward, a_param=a_param, c_param=c_param)
    linaffcont_coup_rand = create_flows_with_alt_identity_IAR(dim_input, dim_hidden, transformations, 'alternate', flow_forward)
    flows[name] += [linaffcont_coup_rand]
    flows[name][-1].name = f'{name} {i}'


In [11]:
#Alt. Lin. Aff.
a_param = torch.sigmoid

name = 'Alternating Linear_Affine lin_reg_exp'
flows[name] = []
for i in range(num_exp):
    transformations = create_alt_linear_affine_trans(num_trans, dim_input, flow_forward,a_param=a_param)
    linaff_coup_rand = create_flows_with_alt_identity_IAR(dim_input, dim_hidden, transformations, 'alternate', flow_forward)
    flows[name] += [linaff_coup_rand]
    flows[name][-1].name = f'{name} {i}'

In [12]:
losses = {n: [] for n in flows.keys()}
optimizers = {n: [] for n in flows.keys()}


for i, flowname in enumerate(flows.keys()):
    for j in range(num_exp):
        # Initialize flow
        flow = flows[flowname][j]
        update_device(device, flow, dataset)

        # Initialize optimizer
        optimizer = torch.optim.AdamW(flow.parameters(), lr=1e-4, weight_decay=1e-2)
        optimizers[flowname].append(optimizer)
        
        # Train and append losses
        losses[flowname].append(
            train_forward(
                flow, 
                flow.base_distr,
                dataset, 
                optimizer, 
                epochs, 
                batch_size,
                batches=batches,
                print_n=200, 
                save_checkpoint=True, 
                burn_in=-1
            )
        )

        # Move flow to CPU
        update_device(device_cpu, flow, dataset)

Finished training. Loss for last epoch Alternating Linear_Affine lin_reg_exp 2:          inf


In [13]:
best_flows = {n:[] for n in flows.keys()}
for flowname in flows.keys():
    for i in range(num_exp):
        best_flows[flowname].append(load_best_model(flows[flowname][i]))
        
dataset.update_device(device_cpu)

In [14]:
flow_elbo_results = {
    'train': {n:[] for n in flows}, 
    'test': {n:[] for n in flows},
}

flow_k_results = {
    'train': {n:[] for n in flows}, 
    'test': {n:[] for n in flows}, 
}

cur_res = 'train'

print('Results based on training data:' + '\n')

# Function for CI
def mean_confidence_interval(data, confidence=0.95):
    import scipy.stats
    a = 1.0 * np.array(data)
    n = len(a)
    m, se = np.mean(a), scipy.stats.sem(a)
    h = se * scipy.stats.t.ppf((1 + confidence) / 2., n-1)
    return m, h
 

for flow in best_flows:
    for i in range(num_exp):
        n_sample = 10000
        cur_flow = best_flows[flow][i]
        update_device(device_cpu, cur_flow, dataset)
        sample, log_flow = cur_flow.sample(n_sample)
        param = sample[-1]
        
        log_target = dataset.evaluate(param, mode=cur_res)
        elbo = torch.mean(log_target - log_flow).detach().numpy()
        flow_elbo_results[cur_res][flow].append(elbo)
        
        _, k_hat, ks = psis_diagnostic(cur_flow, dataset, 10000, mode=cur_res, verbose=False)
        flow_k_results[cur_res][flow].append(k_hat)
        
        print("ELBO for {}: {}".format(f'{flow} {i}', elbo))
        print("Estimated K for {}: {}".format(f'{flow} {i}', k_hat))
        print()
    
    m, h = mean_confidence_interval(flow_elbo_results[cur_res][flow])
    print('-'*75)
    print(f'{flow} -- ELBO Bootstrap: {m:7.4f} ± {h:7.4f}')
    print('-'*75)
    
    m, h = mean_confidence_interval(flow_k_results[cur_res][flow])
    print('-'*75)
    print(f'{flow} -- K_hat Bootstrap: {m:7.4f} ± {h:7.4f}')
    print('-'*75)
    print()

Results based on training data:

ELBO for Mean field lin_reg_exp 0: -12.44987964630127
Estimated K for Mean field lin_reg_exp 0: 0.7587980287829393

ELBO for Mean field lin_reg_exp 1: -12.782971382141113
Estimated K for Mean field lin_reg_exp 1: 0.33813239218174607

ELBO for Mean field lin_reg_exp 2: -12.454035758972168
Estimated K for Mean field lin_reg_exp 2: 0.6255254822080855

---------------------------------------------------------------------------
Mean field lin_reg_exp -- ELBO Bootstrap: -12.5623 ±  0.4748
---------------------------------------------------------------------------
---------------------------------------------------------------------------
Mean field lin_reg_exp -- K_hat Bootstrap:  0.5742 ±  0.5341
---------------------------------------------------------------------------

ELBO for Affine lin_reg_exp iar softplus 0: -13.225189208984375
Estimated K for Affine lin_reg_exp iar softplus 0: 0.6535718016134471

ELBO for Affine lin_reg_exp iar softplus 1: -13.320579

In [22]:
cur_res = 'test'

for flow in best_flows:
    for i in range(num_exp):
        n_sample = 10000
        cur_flow = best_flows[flow][i]
        sample, log_flow = cur_flow.sample(n_sample)
        param = sample[-1]
        
        log_target = dataset.evaluate(param, mode=cur_res)
        elbo = torch.mean(log_target - log_flow).detach().numpy()
        flow_elbo_results[cur_res][flow].append(elbo)
        
        _, k_hat, ks = psis_diagnostic(cur_flow, dataset, 10000, mode=cur_res, verbose=False)
        flow_k_results[cur_res][flow].append(k_hat)
        
        print("ELBO for {}: {}".format(f'{flow} {i}', elbo))
        print("Estimated K for {}: {}".format(f'{flow} {i}', k_hat))
        print()
    
    m, h = mean_confidence_interval(flow_elbo_results[cur_res][flow])
    print('-'*75)
    print(f'{flow} -- ELBO Bootstrap: {m:7.4f} ± {h:7.4f}')
    print('-'*75)
    
    m, h = mean_confidence_interval(flow_k_results[cur_res][flow])
    print('-'*75)
    print(f'{flow} -- K_hat Bootstrap: {m:7.4f} ± {h:7.4f}')
    print('-'*75)
    print()

ELBO for Mean field lin_reg_exp 0: -12.250630378723145
Estimated K for Mean field lin_reg_exp 0: 0.5094214624614534

ELBO for Mean field lin_reg_exp 1: -12.518275260925293
Estimated K for Mean field lin_reg_exp 1: 0.33594965850680736

ELBO for Mean field lin_reg_exp 2: -12.261698722839355
Estimated K for Mean field lin_reg_exp 2: 0.4200494590059354

---------------------------------------------------------------------------
Mean field lin_reg_exp -- ELBO Bootstrap: -12.3435 ±  0.3762
---------------------------------------------------------------------------
---------------------------------------------------------------------------
Mean field lin_reg_exp -- K_hat Bootstrap:  0.4218 ±  0.2155
---------------------------------------------------------------------------

ELBO for Affine lin_reg_exp iar softplus 0: -13.052838325500488
Estimated K for Affine lin_reg_exp iar softplus 0: 0.5999954580628625

ELBO for Affine lin_reg_exp iar softplus 1: -13.124212265014648
Estimated K for Affine